In [ ]:
# Standard imports
import json
from pprint import pprint

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Project imports
import twinlab as tl

In [ ]:
# File paths
campaign_dir = "./../campaigns/basic/"
datasets_dir = "./../datasets/"
dataset = "basic.csv"
file_train = datasets_dir + dataset
file_eval = campaign_dir + "eval.csv"
file_params = campaign_dir + "params.json"

# Campaign id
campaign_id = "basic"

# Server
# server = "local"
server = "cloud"

In [ ]:
# Load the pre-defined parameters file from disk and write to screen
with open(file_params) as f:
    params = json.load(f)
print("Parameters:")
pprint(params)

In [ ]:
# Upload dataset to the cloud
tl.upload_dataset(file_train, server=server, verbose=True)

In [ ]:
# Start a new campaign and train an emulator
tl.train_campaign(params, campaign_id, server=server, verbose=True)

In [ ]:
# Sample from the trained emulator
df_mean, df_std = tl.sample_campaign(file_eval, campaign_id, server=server, verbose=True)

In [ ]:
# List campaigns
tl.list_campaigns(server=server, verbose=True)

In [ ]:
# List datasets
tl.list_datasets(server=server, verbose=True)

In [ ]:
# Load training and evaluation data (for plotting)
df_train = pd.read_csv(file_train)
df_eval = pd.read_csv(file_eval)

In [ ]:
# Plot parameters
nsigs = [1, 2]
# nsigs = [0.674, 1.960, 2.576]
color = "blue"
alpha = 0.5
plot_training_data = True
plot_model_mean = True
plot_model_bands = True

# Plot results
grid = df_eval["x"]
mean = df_mean["y"]
err = df_std["y"]
if plot_model_bands:
    label = r"Model prediction"
    plt.fill_between(grid, np.nan, np.nan, lw=0, color=color, alpha=alpha, label=label)
    for isig, nsig in enumerate(nsigs):
        plt.fill_between(grid, mean-nsig*err, mean+nsig*err, lw=0, color=color, alpha=alpha/(isig+1))
if plot_model_mean:
    label = r"Model prediction" if not plot_model_bands else None
    plt.plot(grid, mean, color=color, alpha=alpha, label=label)
if plot_training_data:
    plt.plot(df_train["x"], df_train["y"], ".", color="black", label="Training data")
plt.xlim((0.0, 1.0))
plt.xlabel(r"$X$")
plt.ylabel(r"$y$")
plt.legend()
plt.show()

In [ ]:
# Delete campaign and dataset (if desired)
tl.delete_campaign(campaign_id, server=server, verbose=True)
tl.delete_dataset(file_train, server=server, verbose=True)